In [ ]:

import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataset

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self,d_k,d_model, n_heads, max_len, causal=False):
    super().__init__()

    self.n_heads=n_heads
    self.d_k=d_k

    self.key=nn.Linear(d_model,d_k * n_heads)
    self.query= nn.Linear(d_model, d_k * n_heads)
    self.value=nn.Linear(d_model, d_k * n_heads)

    self.fc=nn.Linear(d_k * n_heads, d_model)

    self.causal=causal
    if causal:
      cm=torch.tril(torch.ones(max_len,max_len))
      self.register_buffer(
          'causal_mask',
          cm.view(1,1,max_len,max_len)
      )
  def forward(self,q,k,v,pad_mask=False):
    q=self.query(q)
    k=self.key(k)
    v=self.value(v)

    N=q.shape[0]
    T_output=q.shape[1]
    T_input= k.shape[1]

    q=q.view(N,T_output, self.n_heads, self.d_k).transpose(1,2)
    k=k.view(N,T_input, self.n_heads, self.d_k).transpose(1,2)
    v=v.view(N,T_input, self.n_heads, self.d_k).transpose(1,2)

    attn_scores= q @ k.transpose(-2,-1)/ math.sqrt(self.d_k)
    if pad_mask is not None:
      attn_scores=attn_scores.masked_fill(
          pad_mask[:,None,None,:] ==0 ,float('-inf')
      )
    if self.causal:
      attn_scores= attn_scores.masked_fill(
          self.causal_mask[:,:,:T_output, :T_input] == 0, float('-inf')
      )

    attn_weights =F.softmax(attn_scores,dim=-1)

    A=attn_weights @ v
    A=A.transpose(1,2)
    A=A.contiguous().view(N, T_output, self.d_k * self.n_heads)

    return self.fc(A)

In [ ]:
class EncoderBlock(nn.Module):
  def __init__(self, d_k, d_model, n_heads, max_len, dropout_prob=0.1):
    super().__init__()

    self.ln1=nn.LayerNorm(d_model)
    self.ln2=nn.LayerNorm(d_model)
    self.mha=MultiHeadAttention(d_k, d_model, n_heads, max_len, causal=False)
    self.ann= nn.Sequential(
        nn.Linear(d_model, d_model * 4),
        nn.GELU(),
        nn.Linear(d_model * 4, d_model),
        nn.Dropout(dropout_prob)
    )
    self.dropout=nn.Dropout(p=dropout_prob)

  def forward(self,x, pad_mask=None):
    x=self.ln1(x+ self.mha(x,x,x, pad_mask))
    x=self.ln2(x + self.ann(x))
    x=self.dropout(x)

    return x


In [ ]:
class DecoderBlock(nn.Module):
  def __init__(self, d_k, d_model, n_heads, max_len, dropout_prob=0.1):
    super().__init__()

    self.ln1=nn.LayerNorm(d_model)
    self.ln2=nn.LayerNorm(d_model)
    self.ln3=nn.LayerNorm(d_model)
    self.mha1=MultiHeadAttention(d_k,d_model,n_heads,max_len,causal=True)
    self.mha2=MultiHeadAttention(d_k,d_model,n_heads,max_len,causal=False)
    self.ann=nn.Sequential(
        nn.Linear(d_model,d_model * 4),
        nn.GELU(),
        nn.Linear(d_model * 4, d_model),
        nn.Dropout(dropout_prob)
    )
    self.dropout=nn.Dropout(p=dropout_prob)

  def forward(self,enc_output, dec_input, enc_mask=None, dec_mask=None):
    x=self.ln1(dec_input + self.mha1(dec_input,dec_input,dec_input,dec_mask))
    x=self.ln2(x+ self.mha2(x,enc_output,enc_output,enc_mask))
    x=self.ln3(x + self.ann(x))
    x=self.dropout(x)

    return x


In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_len=2048, dropout_prob=0.1):
    super().__init__()
    self.dropout=nn.Dropout(p=dropout_prob)

    position=torch.arange(max_len).unsqueeze(1)
    exp_term=torch.arange(0, d_model,2)
    div_term=torch.exp(exp_term * (-math.log(10000.0)/ d_model))
    pe = torch.zeros(1,max_len,d_model)
    pe[0,:,0::2]=torch.sin(position * div_term)
    pe[0,:,1::2]=torch.cos(position * div_term)
    self.register_buffer('pe',pe)

  def forward(self,x):
    x= x+self.pe[:,:x.size(1), :]
    return self.dropout(x)

In [ ]:
class Encoder(nn.Module):
  def __init__(self, vocab_size, max_len, d_k, d_model, n_heads, n_layers, dropout_prob):
    super().__init__()

    self.embedding=nn.Embedding(vocab_size,d_model)
    self.pos_encoding =PositionalEncoding(d_model, max_len, dropout_prob)
    transformer_blocks=[
        EncoderBlock(d_k, d_model, n_heads, max_len, dropout_prob) for _ in range(n_layers)
    ]
    self.transformer_blocks = nn.Sequential(*transformer_blocks)
    self.ln= nn.LayerNorm(d_model)

  def forward(self, x, pad_mask=None):
    x=self.embedding(x)
    x=self.pos_encoding(x)
    for block in self.transformer_blocks:
      x=block(x,pad_mask)
    x=self.ln(x)
    return x


In [ ]:
class Decoder(nn.Module):
  def __init__(self,vocab_size, max_len, d_k, d_model, n_heads, n_layers, dropout_prob):
    super().__init__()

    self.embedding=nn.Embedding(vocab_size, d_model)
    self.pos_encoding= PositionalEncoding(d_model, max_len, dropout_prob)
    transformer_blocks=[
        DecoderBlock(d_k,d_model,n_heads,max_len, dropout_prob) for _ in range(n_layers)
    ]
    self.transformer_blocks=nn.Sequential(*transformer_blocks)
    self.ln=nn.LayerNorm(d_model)
    self.fc=nn.Linear(d_model,vocab_size)

  def forward(self, enc_output, dec_input,enc_mask=None, dec_mask=None):
    x=self.embedding(dec_input)
    x=self.pos_encoding(x)
    for block in self.transformer_blocks:
      x=block(enc_output, x,enc_mask, dec_mask)
    x=self.ln(x)
    x=self.fc(x)
    return x

In [ ]:
class Transformer(nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder=encoder
    self.decoder=decoder

  def forward(self, enc_input, dec_input,enc_mask, dec_mask):
    enc_output=self.encoder(enc_input,enc_mask)
    dec_output=self.decoder(enc_output, dec_input, enc_mask, dec_mask)
    return dec_output

In [ ]:
encoder=Encoder(vocab_size=20_000,
                max_len=512,
                d_k=16,
                d_model=64,
                n_heads=4,
                n_layers=2,
                dropout_prob=0.1)
decoder=Decoder(vocab_size=10_000,
                max_len=512,
                d_k=16,
                d_model=64,
                n_heads=4,
                n_layers=2,
                dropout_prob=0.1)
transformer=Transformer(encoder,decoder)

In [ ]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
encoder.to(device)
decoder.to(device)

cuda:0


Decoder(
  (embedding): Embedding(10000, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): DecoderBlock(
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln3): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha1): MultiHeadAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (mha2): MultiHeadAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
 

In [ ]:
xe=np.random.randint(0, 20_000, size=(8,512))
xe_t = torch.tensor(xe).to(device)

xd=np.random.randint(0, 10_000, size=(8, 256))
xd_t=torch.tensor(xd).to(device)

maske=np.ones((8,512))
maske[:, 256:]=0
maske_t =torch.tensor(maske).to(device)

maskd=np.ones((8, 256))
maskd[:, 128:]=0
maskd_t=torch.tensor(maskd).to(device)

out=transformer(xe_t,xd_t,maske_t,maskd_t)
out.shape

torch.Size([8, 256, 10000])

In [ ]:
out

tensor([[[ 0.0291,  0.0179, -0.2248,  ...,  0.1798,  0.3970,  0.1240],
         [-0.2569, -0.1338, -0.4435,  ...,  1.2045,  0.6006, -0.8386],
         [-0.5610, -0.0523, -0.4344,  ...,  0.5761,  0.3616, -0.6835],
         ...,
         [ 0.5918, -0.7965,  0.3836,  ..., -1.3303,  0.4669, -0.3869],
         [ 0.9962, -0.0948, -0.6274,  ..., -0.4920,  0.4069, -0.5687],
         [-0.3026,  0.1456, -0.6339,  ...,  0.0570, -0.2363,  0.5978]],

        [[-0.0372, -0.5612, -0.7074,  ..., -0.1086,  1.4962, -0.3696],
         [ 0.0471,  0.0464,  0.4227,  ..., -0.1234,  0.6156,  0.1126],
         [-1.2047, -0.6903, -0.2472,  ...,  0.9168, -0.0991,  0.0356],
         ...,
         [-0.1845,  0.2255, -0.6451,  ...,  0.1203,  0.2932, -0.0535],
         [ 0.7820,  0.2675, -0.2626,  ..., -0.1951,  0.0590,  0.0405],
         [-0.3041, -0.9894,  0.4025,  ..., -0.9052,  0.0596, -0.1135]],

        [[-0.1922, -0.1888, -0.0240,  ...,  1.0259, -0.2711, -0.8096],
         [-0.2616,  0.0299, -0.7159,  ...,  1

In [ ]:
!wget -nc https://lazyprogrammer.me/course_files/nlp3/spa.txt

--2024-03-13 17:07:22--  https://lazyprogrammer.me/course_files/nlp3/spa.txt
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘spa.txt’

spa.txt                 [    <=>             ]   7.45M  7.84MB/s    in 1.0s    

2024-03-13 17:07:24 (7.84 MB/s) - ‘spa.txt’ saved [7817148]



In [ ]:
!head spa.txt

Go.	Ve.
Go.	Vete.
Go.	Vaya.
Hi.	Hola.
Run!	¡Corre!
Who?	¿Quién?
Wow!	¡Órale!
Fire!	¡Fuego!
Fire!	¡Incendio!
Fire!	¡Disparad!


In [ ]:
import pandas as pd
df = pd.read_csv('spa.txt', sep='\t', header=None)
df.head()

,0,1
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Hi.,Hola.
4,Run!,¡Corre!


In [ ]:
df.shape

(115245, 2)

In [ ]:
df=df.iloc[:30_000]

In [ ]:
df.columns=['en','es']
df.to_csv('spa.csv', index=None)

In [ ]:
!head spa.csv

en,es
Go.,Ve.
Go.,Vete.
Go.,Vaya.
Hi.,Hola.
Run!,¡Corre!
Who?,¿Quién?
Wow!,¡Órale!
Fire!,¡Fuego!
Fire!,¡Incendio!


In [ ]:
!pip install transformers datasets sentencepiece sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00


In [ ]:
! pip install datasets

In [ ]:
from datasets import load_dataset
raw_dataset = load_dataset('csv', data_files='spa.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'es'],
        num_rows: 30000
    })
})

In [ ]:
split = raw_dataset['train'].train_test_split(test_size=0.3,seed=42)
split

DatasetDict({
    train: Dataset({
        features: ['en', 'es'],
        num_rows: 21000
    })
    test: Dataset({
        features: ['en', 'es'],
        num_rows: 9000
    })
})

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-es"
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

In [ ]:
en_sentence=split['train'][0]['en']
es_sentence =split['train'][0]['es']

inputs= tokenizer(en_sentence)
targets= tokenizer(text_target=es_sentence)

In [ ]:
tokenizer.convert_ids_to_tokens(targets['input_ids'])

['▁Yo', '▁puedo', '▁arreglarlo', '.', '</s>']

In [ ]:
es_sentence

'Yo puedo arreglarlo.'

In [ ]:
max_input_length=128
max_target_length=128

def preprocess_function(batch):
  model_inputs=tokenizer(
      batch['en'],max_length=max_input_length,truncation=True
  )
  labels=tokenizer(
      text_target=batch['es'],max_length=max_target_length, truncation=True
  )
  model_inputs['labels']=labels['input_ids']
  return model_inputs

In [ ]:
tokenized_datasets=split.map(preprocess_function,batched=True,remove_columns=split['train'].column_names,)

Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 9000
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator=DataCollatorForSeq2Seq(tokenizer)

In [ ]:
batch= data_collator([tokenized_datasets['train'][i] for i in range(0,5)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
batch['input_ids']

tensor([[   33,    88,  9222,    48,     3,     0, 65000, 65000],
        [  552, 11490,     9,   310,   255,     3,     0, 65000],
        [  143,    31,   125,  1208,     3,     0, 65000, 65000],
        [ 1093,   220,  1890,    23,    48,     3,     0, 65000],
        [  124,    20,   100, 18422,    48,   141,     3,     0]])

In [ ]:
batch['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])

In [ ]:
batch['labels']

tensor([[  711,  1039, 44159,     3,     0,  -100,  -100,  -100],
        [ 2722, 18663,   239,   212,     3,     0,  -100,  -100],
        [  539,    43,   155,   960,     3,     0,  -100,  -100],
        [15165,  1250,   380,  3564,    36,  1016,     3,     0],
        [  350,     8, 19153,    29, 31326,     3,     0,  -100]])

In [ ]:
tokenizer.all_special_ids

[0, 1, 65000]

In [ ]:
tokenizer.all_special_tokens

['</s>', '<unk>', '<pad>']

In [ ]:
tokenizer('<pad>')

{'input_ids': [65000, 0], 'attention_mask': [1, 1]}

In [ ]:
from torch.utils.data import DataLoader

train_loader= DataLoader(
    tokenized_datasets['train'],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
valid_loader=DataLoader(
    tokenized_datasets['test'],
    batch_size=32,
    collate_fn=data_collator
)

In [ ]:
for batch in train_loader:
  for k,v in batch.items():
    print('K: ',k,'v.shape: ',v.shape)
  break

K:  input_ids v.shape:  torch.Size([32, 10])
K:  attention_mask v.shape:  torch.Size([32, 10])
K:  labels v.shape:  torch.Size([32, 8])


In [ ]:
tokenizer.vocab_size

65001

In [ ]:
tokenizer.decode([60000])

'ѕэр'

In [ ]:
tokenizer.add_special_tokens({'cls_token': '<s>'})

1

In [ ]:
tokenizer('<s>')

{'input_ids': [65001, 0], 'attention_mask': [1, 1]}

In [ ]:
tokenizer.vocab_size

65001

In [ ]:
encoder=Encoder(vocab_size=tokenizer.vocab_size+1,
                max_len=512,
                d_k=16,
                d_model=64,
                n_heads=4,
                n_layers=2,
                dropout_prob=0.1)
decoder=Decoder(vocab_size=tokenizer.vocab_size+1,
                max_len=512,
                d_k=16,
                d_model=64,
                n_heads=4,
                n_layers=2,
                dropout_prob=0.1)
transformer=Transformer(encoder,decoder)

In [ ]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
encoder.to(device)
decoder.to(device)

cuda:0


Decoder(
  (embedding): Embedding(65002, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): DecoderBlock(
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln3): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha1): MultiHeadAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (mha2): MultiHeadAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
 

In [ ]:
criterion=nn.CrossEntropyLoss(ignore_index=-100)
optimizer=torch.optim.Adam(transformer.parameters())

In [ ]:
from datetime import datetime

def train(model, criterionm, optimizer, train_loader, valid_loader, epochs):
  train_losses = np.zeros(epochs)
  test_losses= np.zeros(epochs)

  for it in range(epochs):
    model.train()
    t0 = datetime.now()
    train_loss =[]
    for batch in train_loader:
      batch={k: v.to(device) for k,v in batch.items()}

      optimizer.zero_grad()
      enc_input=batch['input_ids']
      enc_mask=batch['attention_mask']
      targets=batch['labels']

      dec_input= targets.clone().detach()
      dec_input= torch.roll(dec_input, shifts=1,dims=1)
      dec_input[:,0]=65_001

      dec_input= dec_input.masked_fill(
          dec_input == -100, tokenizer.pad_token_id
      )

      dec_mask = torch.ones_like(dec_input)
      dec_mask= dec_mask.masked_fill(dec_input == tokenizer.pad_token_id, 0)

      outputs = model(enc_input, dec_input, enc_mask, dec_mask)
      loss= criterion(outputs.transpose(2,1), targets)

      loss.backward()
      optimizer.step()
      train_loss.append(loss.item())

    train_loss= np.mean(train_loss)

    model.eval()
    test_loss=[]
    for batch in valid_loader:
      batch={k: v.to(device) for k,v in batch.items()}

      enc_input =batch['input_ids']
      enc_mask = batch['attention_mask']
      targets = batch['labels']

      dec_input = targets.clone().detach()
      dec_input= torch.roll(dec_input, shifts=1, dims=1)
      dec_input[:,0]= 65_001

      dec_input = dec_input.masked_fill(
          dec_input == -100, tokenizer.pad_token_id
      )

      dec_mask = torch.ones_like(dec_input)
      dec_mask = dec_mask.masked_fill(dec_input == tokenizer.pad_token_id, 0)

      outputs =model(enc_input,dec_input,enc_mask, dec_mask)
      loss= criterion(outputs.transpose(2,1), targets)
      test_loss.append(loss.item())

    test_loss=np.mean(test_loss)

    train_losses[it]=train_loss
    test_losses[it]= test_loss

    dt = datetime.now() - t0
    print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Duration: {dt}')

  return train_losses, test_losses

In [ ]:
train_losses, test_losses = train(
    transformer, criterion, optimizer, train_loader, valid_loader, epochs= 15
)

Streaming output truncated to the last 5000 lines.
        [65001,   107, 26405,    28,  5770,   485,    55,     0, 65000],
        [65001,  4355, 47292,     3,     0, 65000, 65000, 65000, 65000],
        [65001,  2634,  2193,     3,     0, 65000, 65000, 65000, 65000],
        [65001,  1782,    43, 12610,     3,     0, 65000, 65000, 65000],
        [65001,   107,  2229,  1726,     8,  7149,    55,     0, 65000],
        [65001,    54,  5669,   108, 29133,     3,     0, 65000, 65000]],
       device='cuda:0')
Traget : tensor([[21958,  4833,  9360,     6,   992,     3,     0,  -100,  -100],
        [  649,  9863,   108, 14677,     3,     0,  -100,  -100,  -100],
        [   71,    57,  1534,  8374,     3,     0,  -100,  -100,  -100],
        [ 4963,   223,  9326,    29,  5266,     3,     0,  -100,  -100],
        [   50,  1102,   694,   357,    21,     0,  -100,  -100,  -100],
        [   71,   157,   135, 24861,   152,   293,     3,     0,  -100],
        [  377,  2357,     6, 30830,   

In [ ]:
input_sentence = split['test'][10]['en']
input_sentence

'Can I take a day off?'

In [ ]:
enc_input= tokenizer(input_sentence, return_tensors='pt')
enc_input

{'input_ids': tensor([[1283,   33,  273,    8,  502,  843,   21,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
dec_input_str='<s>'

dec_input= tokenizer(text_target=dec_input_str, return_tensors='pt')
dec_input

{'input_ids': tensor([[65001,     0]]), 'attention_mask': tensor([[1, 1]])}

In [ ]:
enc_input.to(device)

{'input_ids': tensor([[1283,   33,  273,    8,  502,  843,   21,    0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
dec_input.to(device)
output = transformer(
    enc_input['input_ids'],
    dec_input['input_ids'][:,:-1],
    enc_input['attention_mask'],
    dec_input['attention_mask'][:,:-1]
)
output

tensor([[[  5.7652, -13.2062,   3.4866,  ..., -14.0792, -10.6156, -13.0982]]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [ ]:
output.shape

torch.Size([1, 1, 65002])

In [ ]:
enc_output= encoder(enc_input['input_ids'], enc_input['attention_mask'])
enc_output.shape

torch.Size([1, 8, 64])

In [ ]:
dec_output=decoder(
    enc_output,
    dec_input['input_ids'][:,:-1],
    enc_input['attention_mask'],
    dec_input['attention_mask'][:,:-1]
)
dec_output.shape

torch.Size([1, 1, 65002])

In [ ]:
torch.allclose(output,dec_output)

True

In [ ]:
dec_input_ids = dec_input['input_ids'][:,:-1]
dec_attn_mask =dec_input['attention_mask'][:,:-1]

for _ in range(32):
  dec_output=decoder(
      enc_output,
      dec_input_ids,
      enc_input['attention_mask'],
      dec_attn_mask,
  )

  prediction_id = torch.argmax(dec_output[:, -1, :], axis=-1)
  dec_input_ids= torch.hstack((dec_input_ids, prediction_id.view(1,1)))
  dec_attn_mask= torch.ones_like(dec_input_ids)

  if prediction_id == 0:
    break


In [ ]:
tokenizer.decode(dec_input_ids[0])

'<s> ¿Puedo tomar un día de día?</s>'

In [ ]:
split['test'][10]['es']

'¿Puedo tomarme un día libre?'

In [ ]:
def translate(input_sentence):
  enc_input = tokenizer(input_sentence, return_tensors='pt').to(device)
  enc_output= encoder(enc_input['input_ids'], enc_input['attention_mask'])

  dec_input_ids = torch.tensor([[65_001]], device=device)
  dec_attn_mask = torch.ones_like(dec_input_ids, device=device)

  for _ in range(32):
    dec_output=decoder(
        enc_output,
        dec_input_ids,
        enc_input['attention_mask'],
        dec_attn_mask,
    )

    prediction_id=torch.argmax(dec_output[:,-1,:], axis=- 1)
    dec_input_ids = torch.hstack((dec_input_ids,prediction_id.view(1,1)))
    dec_attn_mask=torch.ones_like(dec_input_ids)

    if prediction_id == 0:
      break

  translation = tokenizer.decode(dec_input_ids[0, 1:])
  print(translation)

In [ ]:
translate('How are you?')

¿Cómo están?</s>
